# This is an implementation for computing Collision Free Regions in Configuration Space using IRIS

[IRIS in Configuration Space](https://drake.mit.edu/doxygen_cxx/group__geometry__optimization.html#ga3a51e0fec449a0abcf498f78a2a390a8)


This notebook contains code for creating, visualizing and saving such regions.

In [ ]:
import sys
import os

if 'google.colab' in sys.modules:
  print("You are in colab")
  from google.colab import drive
  drive.mount('/content/drive')
  os.chdir('/content/drive/MyDrive/Otros/arms-robotic-system/jupyter_notebooks')
  !pip install pyngrok drake
  from pyngrok import ngrok

from pydrake.geometry import ( StartMeshcat )
from pydrake.geometry.optimization import (SaveIrisRegionsYamlFile) # type: ignore

# add parent folders to path
sys.path.append(os.path.abspath('..'))

# Update the import
from src.auxiliar_functions import (VisualizeConnectivity)
from src.gcs_trajectory_optimization import GcsTrajOpt
from src.iris_options import load_config, create_iris_options
from src.joint_teleop import JointTeleop
from src.gcs_helper import VisualizeRegions, GenerateRegions
from src.end_effector_teleop import EndEffectorTeleop

# Start the visualizer (only run this once).
meshcat = StartMeshcat()
if 'google.colab' in sys.modules:
  # Create a tunnel to the Meshcat server
  ngrok.set_auth_token("2rdws4UBYTqZ1NiiyEbJcWRnByV_7D9Jg8K3YVyZzbFMN1ysg")
  public_url = ngrok.connect(7000, "http")
  print(f"Meshcat is accessible at: {public_url}")

# Setup 



In [ ]:
# Get the absolute path of the current working directory
parent_dir = os.path.dirname(os.getcwd())
iris_filename = os.path.join(parent_dir,'common-files','regions','my_regions2.yaml')

config = load_config(os.path.join(parent_dir,'common-files','iris_config.yaml'))
iris_options = create_iris_options(config)

seeds_joint_teleop = []
iris_regions = {}

In [ ]:
import json
import numpy as np

# Load joint positions from a JSON file
with open('/home/arath/github/arms-robotic-system/common-files/gcs_seeds.json', 'r') as file:
    joint_positions_data = json.load(file)

# Convert the lists to numpy arrays
joint_positions = {key: np.array(value) for key, value in joint_positions_data.items()}

In [ ]:
GenerateRegions(joint_positions, config, iris_regions,iris_options)
iris_regions = {key: value for key, value in iris_regions.items() if value is not None}

## Run IRIS on manually-specified seeds

In [ ]:
print(iris_regions.keys())
#VisualizeRegions(meshcat, iris_regions)

#### End Effector Teleoperation.

In [ ]:
#EndEffectorTeleop(meshcat, iris_regions, config, seeds_joint_teleop, iris_options) # End effector teleoperation

## Joint Teleop

In [ ]:
JointTeleop(meshcat, seeds_joint_teleop, iris_regions, config) # Joint teleoperation

## Visualize the current IRIS region connectivity

For planning trajectories, it's important to have some non-empty intersection between the convex sets. Here is a simple utility the visualization the connectivity as a graph.

In [ ]:
VisualizeConnectivity(iris_regions)

## Plan trajectories with GCS Trajectory Optimization

# Save your regions back to disk

Note that by default, this notebook will autosave a backup after making each region.

In [ ]:
SaveIrisRegionsYamlFile(iris_filename, iris_regions)

In [ ]:
# clear the meshcat visualizer
meshcat.Delete()
meshcat.DeleteAddedControls()

#GcsTrajOpt(joint_positions["ketchup_in_front"], joint_positions["right_bin_bottom_rack"],iris_regions, meshcat)